In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import HTML
with open("rise.css", "r") as f:
    s = f"<style>{f.read()}</style>"
HTML(s)

In [ ]:
import plotly.graph_objects as go
import numpy as np
from not_rocketscience import config
from copy import deepcopy
import plotly.io as pio
my_template = deepcopy(pio.templates["plotly_dark"])
my_template["layout"]["colorway"] = [config.colors.colors["ship_red"], config.colors.colors["ship_blue"], config.colors.colors["ship_green"]]
my_template["layout"]["paper_bgcolor"] = config.colors.colors["space"]
my_template["layout"]["plot_bgcolor"] = config.colors.colors["space"]
my_template["layout"]["font"]["color"] = config.colors.colors["stars"]
pio.templates["notrocket"] = my_template
pio.templates.default = "notrocket"

<div class="title">

<h1 class="title">Not Rocketscience</h1>

## A spaceship-flying game built with pygame

### Andreas Roth - 13. June 2024

</div>

# 2D pixel graphics
    
Originating from of technical restriction, they are today just adorable to look at.

<div class="row">
<div class="column" style="width:33%">
<img src="assets/pixel_andi.png">
</div>
<div class="column" style="width:33%">
<img src="assets/pixel_python.png">
</div> 
<div class="column" style="width:33%">
<img src="assets/pixel_computer.png">
</div>
</div>

Let's make the pictures move!

<div class="sect">
    
## What is pygame for?
    
* [pygame](https://www.pygame.org/news) is a python binding for the [SDL (Simple DirectMedia Layer)](https://www.libsdl.org/) C-library
* Pygame makes it very easy to render 2D graphics and geometry to the screen and move them around.
* Additionally, keyboard and controller input can be easily processed
* Sounds can be played, too (out of scope for this talk)
    
</div>



## How does pygame do that?

<img src="assets/very_simple.png" width="450px">

* We define ``pygame.Surface``s 
    * geom. forms,
    * image files, 
    * the ``screen`` itself 

    and display them at pixel positions ``(x, y)``
* Change positions over time $
\rightarrow$ movement
* React to inputs $\rightarrow$ video game

## Program structure

**Game loop**: Not a script that runs from beginning to end, but rather runs continuously and reacts on **events**

```python
running = True
while running:
    process_inputs()  # react on inputs
    render_scene()  # place elements on the screen
    update_display()  # update what is displayed
    # control how long this takes!
    if exit_clicked():
        running = False
```
One iteration of the game loop defines one **frame** of the game.
    

# Let's dive right in!

Take a look at [``very_basic.py``](./very_basic.py), and the cod eit is based on [``GameBase``](../src/not_rocketscience/framework.py) and add some input processing!

<div class="sect">
    
## How to build planets
    
* We can draw filled 2D circles
* We can draw rectangular pixel images (with alpha channel for transparency)
* How to create a "rotating" planet?
    
</div>

## How to build planets

<img src="assets/planet_disc_1.png" width="500px">

**Step 1:** White circle on black square

<img src="assets/planet_disc_2.png" width="500px">

**Step 2:** Set the alpha channel of the background to 0 before drawing a circle with alpha value 255

## How to build planets

**Step 3:** Load a square planet texture 
<img src="../src/not_rocketscience/assets/planets/planet_02.png" width="700px">



**Step 4:** Multiply color values of white circle and black background with texture
<img src="assets/planet_disc_3.png" width="400px">

**Step 5:** layer some transparent circles behind the planet disc to get a nice atmosphere.

<img src="assets/planet_disc_4.png" width="700px">



# How to make the planets rotate?

Take a look at [``planet_animation.py``](./planet_animation.py), and the code it is based on [``PlanetTexture``](../src/not_rocketscience/planets.py)!


<div class="sect">
    
## Movement and mechanics: Newtonian dynamics!
    
* As long as no forces act on an object, it does not change its velocity
* Forces change the velocity of an object, and velocity changes the position
* [Newton's laws of motion on Wikipedia](https://en.wikipedia.org/wiki/Newton%27s_laws_of_motion)
    
</div>



(Placeholder for Sir Isaac Newton)

## The laws of motion as equations

Let $x$ be a position, $v$ speed, $a$ acceleration, $F$ a force and $m$ the mass of an object

<div class="row">
<div class="column">
        
$$
\frac{dx}{dt} = v
$$
    
</div>
<div class="column">

Change of position over time is given by velocity

</div>
</div>
<div class="row">
<div class="column">

$$
\frac{dv}{dt} = a = \frac{F}{m}
$$

</div>
<div class="column">

Change of velocity over time is acceleration which is force divided by mass

</div>
</div>

Of course, $x, v, a, F$ are 2D vectors because we move around on a plane, the screen!


### How do we calculate the change in position of an object?

<div class="row">
<div class="column" style="width:30%">
<img src="assets/ship_orientation.png" width="400px">
</div>
<div class="column" style="width:70%">

Apply thrust into the direction into which your ship points, how does the position change?
$$
a_t = \begin{pmatrix}\cos{\alpha}&-\sin{\alpha}\\\sin{\alpha}&\cos{\alpha}\end{pmatrix}\cdot\begin{pmatrix}0\\T_t\end{pmatrix} - d\cdot v_t
$$
where $T_t$ is thrust applied (pixels per second per second) at time $t$, $d$ a damping factor (like air drag)
    
\begin{align}
v_{t+\Delta t} &= v_t + \Delta t\cdot a_t\\
x_{t+\Delta t} &= x_t + \Delta t\cdot v_t
\end{align}
    
This is actually a proper numerical solution to Newtons laws of motion, called the ["Leapfrog scheme"](https://en.wikipedia.org/wiki/Leapfrog_integration)
</div>
</div>

# Let's try that out!

* Run [``run_main.py``](./run_main.py) and fly around a bit!
* Rotate the ship with the arrow keys left and right and change $\alpha$
* Press the X key to apply thrust and set $T_t$ to a constant value while the X key is held down. Otherwise it is zero.
* What happens around planets?

<div class="sect">

## Gravity calculations
    
* Something is off with that gravity? Well, it's not Newton's gravity
* A special gravity force has been "designed" for this game, to make it easier to handle and more fun!
    
</div>

## Newtonian gravity is not fun to play with
[Classic law of gravity](https://en.wikipedia.org/wiki/Newton%27s_law_of_universal_gravitation): Two objects attract each other depending on their distance and masses.

* Let $x$ be the position of a test particle (our ship) with unit mass, let $x_P$ be a gravitational source (a planet),
* $|x - x_P|$ denotes the distance between ship and planet, 

then it holds for the gravitational force on $x$, $F(x)$

\begin{align}
F(x) &= \underbrace{-\frac{Gm_p}{|x-x_p|^2}}_{\text{force magnitude}} \cdot \underbrace{\frac{x-x_p}{|x-x_p|}}_{\text{unit vector}}
\end{align}

In [ ]:
from not_rocketscience.math import newton_gravity_force, newton_gravity_force_derivative, canonical_newton_parameterset, newton_iteration

scale = 600 / newton_iteration(
    lambda r: newton_gravity_force(*canonical_newton_parameterset)(r) + 355,
    newton_gravity_force_derivative(*canonical_newton_parameterset),
    initial_value=0.03, tolerance=0.001)
xn = np.linspace(0, 1000, 100)
go.Figure(
    data=[go.Scatter(name="Newton force", x=xn, y=newton_gravity_force(*canonical_newton_parameterset)(xn / scale))],
    layout=dict(title="Newton gravity force magnitude (cut off for small values)", xaxis=dict(title="distance in pixels"), yaxis=dict(title="force magnitude px per second per second"))
)

## Introducing the "Weird Gravity"

In [ ]:
from not_rocketscience.math import weird_gravity_force, canonical_weird_parameterset, weird_gravity_force_derivative, weird_gravity_potential

scale = 200 / newton_iteration(
    weird_gravity_force(*canonical_weird_parameterset),
    weird_gravity_force_derivative(*canonical_weird_parameterset),
    initial_value=0, tolerance=1.0e-3
)

x = np.linspace(0, 1000, 100)
go.Figure(
    data=[
        go.Scatter(name="Force", x=x, y=weird_gravity_force(*canonical_weird_parameterset)(x / scale)),
        go.Scatter(name="Potential", x=x, y=weird_gravity_potential(*canonical_weird_parameterset)(x / scale))
    ],
    layout=dict(title="Gravity potential and force over distance", xaxis=dict(title="distance"))
)


* We want finite force values if distance approaches zero.
* We do not want collisions with planets, so when you get to close, the weird gravity pushes you away again.

Behold our force magnitude $F(r)$ ($r$ being the distance)

\begin{align}
\Phi(r) &= 4 \cdot \log(6(r + 0.2)) \cdot \exp(-4 (r - 1.2)^2) \\
F(r) &= 4 \cdot \exp(-4 (r - 1.2)^2)\\
&\cdot(\frac{1}{r + 0.2} - 4\log(6(r + 0.2)))
\end{align}

### Performance considerations

Each planet makes a contribution to the gravity force on our ship, so **in every frame**, we have to calculate for $N$ planets

$$
F_{total}(x) = \sum_{i=1}^N F(|x - x_i|) \cdot\frac{x-x_i}{|x-x_i|}
$$

That is inefficient! So we use a KDTree with the world coordinates of the planets

## World coordinates

<div class="row">
<div class="column">
<img src="assets/world_coordinates.jpg" width="90%">
</div>
<div class="column">
    
* ship moves
* planets don't move
    
</div>
</div>

## Screen coordinates

<div class="row">
<div class="column">
<img src="assets/screen_coordinates.jpg" width="90%">
</div>
<div class="column">
    
* ship does not move
* planets move
    
</div>
</div>